# Notebook for data parsing 

In [1]:
# Mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

# Data managment 
import pandas as pd 
import numpy as np 
from sklearn import preprocessing

#Additional
import pickle
import os

data_folder = '/content/gdrive/My Drive/ColabNotebooks/Data/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Get RAW Data
%%capture
if not os.path.isfile('trending_merged_LHC18f_withGraphs.csv'):
    if not os.path.isfile('CERNdata.zip'):
        !wget 'https://cernbox.cern.ch/index.php/s/JnxbQZ0nzITGC4i/download' -O CERNdata.zip
    !7z x CERNdata.zip

##Show Raw Data

In [0]:
#Proton-Proton
raw_f = pd.read_csv('trending_merged_LHC18f_withGraphs.csv')
raw_f['period'] = 'LHC18f'
raw_o = pd.read_csv('trending_merged_LHC18o_withGraphs.csv')
raw_o['period'] = 'LHC18o'
raw_p = pd.read_csv('trending_merged_LHC18p_withGraphs.csv')
raw_p['period'] = 'LHC18p'

In [4]:
raw_full = pd.concat([raw_f, raw_o, raw_p])
raw_full.head()

,Unnamed: 0,run,chunkID,time,year,period.fString,pass.fString,dataType.fString,startTimeGRP,stopTimeGRP,duration,bz,runType.fString,meanTPCnclF,rmsTPCnclF,meanTPCChi2,rmsTPCChi2,slopeATPCnclF,slopeCTPCnclF,slopeATPCnclFErr,slopeCTPCnclFErr,meanTPCncl,rmsTPCncl,slopeATPCncl,slopeCTPCncl,slopeATPCnclErr,slopeCTPCnclErr,hasRawQA,rawClusterCounter,rawSignalCounter,offsetdZA,slopedZA,offsetdZC,slopedZC,offsetdZAErr,slopedZAErr,offsetdZCErr,slopedZCErr,offsetdZAchi2,slopedZAchi2,...,grdcaz_neg_CSidePhi_Y_106,grdcaz_neg_CSidePhi_Y_107,grdcaz_neg_CSidePhi_Y_108,grdcaz_neg_CSidePhi_Y_109,grdcaz_neg_CSidePhi_Y_110,grdcaz_neg_CSidePhi_Y_111,grdcaz_neg_CSidePhi_Y_112,grdcaz_neg_CSidePhi_Y_113,grdcaz_neg_CSidePhi_Y_114,grdcaz_neg_CSidePhi_Y_115,grdcaz_neg_CSidePhi_Y_116,grdcaz_neg_CSidePhi_Y_117,grdcaz_neg_CSidePhi_Y_118,grdcaz_neg_CSidePhi_Y_119,grdcaz_neg_CSidePhi_Y_120,grdcaz_neg_CSidePhi_Y_121,grdcaz_neg_CSidePhi_Y_122,grdcaz_neg_CSidePhi_Y_123,grdcaz_neg_CSidePhi_Y_124,grdcaz_neg_CSidePhi_Y_125,grdcaz_neg_CSidePhi_Y_126,grdcaz_neg_CSidePhi_Y_127,grdcaz_neg_CSidePhi_Y_128,grdcaz_neg_CSidePhi_Y_129,grdcaz_neg_CSidePhi_Y_130,grdcaz_neg_CSidePhi_Y_131,grdcaz_neg_CSidePhi_Y_132,grdcaz_neg_CSidePhi_Y_133,grdcaz_neg_CSidePhi_Y_134,grdcaz_neg_CSidePhi_Y_135,grdcaz_neg_CSidePhi_Y_136,grdcaz_neg_CSidePhi_Y_137,grdcaz_neg_CSidePhi_Y_138,grdcaz_neg_CSidePhi_Y_139,grdcaz_neg_CSidePhi_Y_140,grdcaz_neg_CSidePhi_Y_141,grdcaz_neg_CSidePhi_Y_142,grdcaz_neg_CSidePhi_Y_143,interactionRate,period
0,0,287000,1,1527204124,2018,LHC18f,pass1,NaN,1527204124,1527231797,27673,-0.499998,PHYSICS,0.913711,0.091254,1.767776,0.446999,-0.003785,0.000947,0.000674,0.000947,133.596942,22.675932,-0.061015,-0.029429,0.172744,-0.029429,1,367302864,547136439,0.008433,0.061533,-0.095885,0.060379,0.002402,0.005589,0.002441,0.005575,54.538913,54.538913,...,-0.052315,-0.058496,-0.057705,-0.007200,-0.066442,-0.079369,-0.117361,-0.061361,-0.099326,-0.078831,-0.097516,-0.107208,-0.109287,-0.102642,-0.066415,-0.069376,-0.061681,-0.096441,-0.055511,-0.059102,-0.068315,-0.102027,-0.086352,-0.070171,-0.066603,-0.059391,-0.111429,-0.073658,-0.090766,-0.029640,-0.084562,-0.079724,-0.069284,-0.045167,-0.065301,-0.050480,-0.050551,-0.069094,195584.670984,LHC18f
1,1,287000,2,1527204124,2018,LHC18f,pass1,NaN,1527204124,1527231797,27673,-0.499998,PHYSICS,0.913795,0.091471,1.765181,0.446966,-0.004339,0.000274,0.000660,0.000274,133.604316,22.668970,-0.227194,-0.046269,0.170329,-0.046269,1,367302864,547136439,0.007302,0.050437,0.041850,0.239221,0.002347,0.005482,0.002514,0.005686,80.242746,80.242746,...,-0.013811,-0.043341,-0.036767,-0.008760,-0.059732,-0.010744,-0.024306,-0.056385,-0.063628,-0.059174,-0.067915,-0.013919,-0.018712,-0.042576,-0.029183,-0.065051,-0.074004,-0.062315,-0.098015,-0.032065,-0.039085,-0.037202,-0.020365,-0.052420,-0.092947,-0.017293,-0.044535,-0.009809,-0.026729,-0.048561,-0.006055,-0.034508,-0.051278,-0.038319,0.017724,0.012698,-0.032471,-0.020261,195584.670984,LHC18f
2,2,287000,3,1527204124,2018,LHC18f,pass1,NaN,1527204124,1527231797,27673,-0.499998,PHYSICS,0.913980,0.091291,1.766238,0.447530,-0.003400,0.000098,0.000655,0.000098,133.619454,22.658997,-0.226418,-0.220931,0.169301,-0.220931,1,367302864,547136439,0.018170,0.035006,0.122846,0.356713,0.002312,0.005374,0.002513,0.005630,84.160363,84.160363,...,-0.005258,0.001520,-0.004679,0.067023,0.020974,-0.022892,-0.021963,-0.054784,-0.065607,-0.046039,-0.033981,-0.057274,-0.023231,-0.018649,0.002054,-0.031758,-0.085111,-0.042000,0.024556,-0.053818,-0.004125,-0.005745,0.006034,-0.042845,-0.028715,0.006710,-0.015597,-0.024007,-0.001764,-0.029742,-0.010322,-0.021278,0.031273,0.034793,0.030032,0.017758,-0.000799,0.018394,195584.670984,LHC18f
3,3,287000,4,1527204124,2018,LHC18f,pass1,NaN,1527204124,1527231797,27673,-0.499998,PHYSICS,0.913710,0.091409,1.769212,0.448276,-0.003525,-0.000843,0.000665,-0.000843,133.561797,22.654738,-0.160837,-0.230723,0.169628,-0.230723,1,367302864,547136439,0.010970,0.044343,0.186074,0.444474,0.0

##Parse away basic trash

In [5]:
#Define meta data, graphs, aliases, no variation cols
meta_cols = ['period', 'run', 'chunkID', 'time', 'year', 'period.fString', 'pass.fString', 
'runType.fString', 'startTimeGRP', 'stopTimeGRP', 'duration', 'chunkStart', 
'chunkStop', 'chunkMean', 'chunkMedian', 'chunkRMS', 
'chunkDuration'] + ['Unnamed: 0', 'dataType.fString' ] 

feat_w_no_variation = ['offsetdRCErr' , 'slopedRCErr', 'offsetdRCErrPos', 'slopedRCErrPos', 'offsetdRCErrNeg', 'slopedRCErrNeg']

graph_cols = [col for col in raw_full if col.startswith('gr')] 
alias_cols = [col for col in raw_full if col.startswith('alias_')] 

#Point to label columns
war_col = 'alias_global_Warning'
out_col = 'alias_global_Outlier'

# Filter out data not used for classification.
data = raw_full.drop(meta_cols + graph_cols + alias_cols + feat_w_no_variation, axis=1)
no_samples, no_features = data.shape

#Create vector with labels - WARNINGS ARE LABELS
labels = raw_full[war_col]
chunkID = raw_full['chunkID']
run = raw_full['run']
period = raw_full['period']

#Check shape
print(no_samples, no_features)
data.head()

3429 233


,bz,meanTPCnclF,rmsTPCnclF,meanTPCChi2,rmsTPCChi2,slopeATPCnclF,slopeCTPCnclF,slopeATPCnclFErr,slopeCTPCnclFErr,meanTPCncl,rmsTPCncl,slopeATPCncl,slopeCTPCncl,slopeATPCnclErr,slopeCTPCnclErr,hasRawQA,rawClusterCounter,rawSignalCounter,offsetdZA,slopedZA,offsetdZC,slopedZC,offsetdZAErr,slopedZAErr,offsetdZCErr,slopedZCErr,offsetdZAchi2,slopedZAchi2,offsetdZCchi2,slopedZCchi2,offsetdZAPos,slopedZAPos,offsetdZCPos,slopedZCPos,offsetdZAErrPos,slopedZAErrPos,offsetdZCErrPos,slopedZCErrPos,offsetdZAchi2Pos,slopedZAchi2Pos,...,oroc_C_side,MIPattachSlopeC,MIPattachSlopeA,resolutionMIP,meanMIP,meanMIPele,resolutionMIPele,electroMIPSeparation,tpcItsMatchA,tpcItsMatchHighPtA,tpcItsMatchC,tpcItsMatchHighPtC,phiPull,phiPullHighPt,ptPull,ptPullHighPt,yPull,yPullHighPt,zPull,zPullHighPt,lambdaPull,lambdaPullHighPt,tpcConstrainPhiA,tpcConstrainPhiC,meanPTRelativeA,medianPTRelativeA,rmsPTRelativeA,meanPTRelativeC,medianPTRelativeC,rmsPTRelativeC,meanHVandPTGainCorrIROC,medianHVandPTGainCorrIROC,rmsHVandPTGainCorrIROC,meanHVandPTGainCorrOROC,medianHVandPTGainCorrOROC,rmsHVandPTGainCorrOROC,meanVDriftCorr,medianVDriftCorr,rmsVDriftCorr,interactionRate
0,-0.499998,0.913711,0.091254,1.767776,0.446999,-0.003785,0.000947,0.000674,0.000947,133.596942,22.675932,-0.061015,-0.029429,0.172744,-0.029429,1,367302864,547136439,0.008433,0.061533,-0.095885,0.060379,0.002402,0.005589,0.002441,0.005575,54.538913,54.538913,1447.258711,1447.258711,0.011558,0.084150,-0.093813,0.087700,0.003431,0.008177,0.003531,0.008089,9.050547,9.050547,...,18,0.030036,-0.173266,0.064220,49.792557,80.735855,0.055183,30.943298,0.699384,0.710692,0.694623,0.703207,-0.000119,-0.000389,0.527906,0.738438,-0.092014,-0.210392,-0.088328,-0.156770,0.119385,0.068962,0.017818,-0.049315,0.003437,0.003433,0.000011,0.003560,0.003556,0.000011,1.128413,1.128399,0.000043,1.128413,1.128399,0.000043,-0.027221,-0.027222,3.005739e-06,195584.670984
1,-0.499998,0.913795,0.091471,1.765181,0.446966,-0.004339,0.000274,0.000660,0.000274,133.604316,22.668970,-0.227194,-0.046269,0.170329,-0.046269,1,367302864,547136439,0.007302,0.050437,0.041850,0.239221,0.002347,0.005482,0.002514,0.005686,80.242746,80.242746,2298.826852,2298.826852,0.006364,0.079816,0.040957,0.262827,0.003490,0.008128,0.003665,0.008344,35.407343,35.407343,...,18,-0.046077,-0.174695,0.064034,49.808353,80.820270,0.055974,31.011913,0.698263,0.696498,0.692948,0.703005,-0.000016,0.001378,0.533350,0.784235,-0.053545,-0.088791,-0.077231,-0.174282,0.118407,0.125998,0.008589,-0.031984,0.003476,0.003474,0.000017,0.003597,0.003596,0.000017,1.128563,1.128559,0.000066,1.128563,1.128559,0.000066,-0.027226,-0.027226,8.138511e-07,195584.670984
2,-0.499998,0.913980,0.091291,1.766238,0.447530,-0.003400,0.000098,0.000655,0.000098,133.619454,22.658997,-0.226418,-0.220931,0.169301,-0.220931,1,367302864,547136439,0.018170,0.035006,0.122846,0.356713,0.002312,0.005374,0.002513,0.005630,84.160363,84.160363,2689.649719,2689.649719,0.018739,0.060694,0.122070,0.383957,0.003383,0.007934,0.003700,0.008220,64.929950,64.929950,...,18,-0.040411,-0.132918,0.063939,49.698433,80.666520,0.055939,30.968086,0.700643,0.707197,0.693301,0.706140,-0.000104,0.000041,0.534116,0.760895,-0.058528,-0.119211,-0.078093,-0.172614,0.118760,0.088854,0.012813,-0.016630,0.003457,0.003462,0.000018,0.003578,0.003583,0.000018,1.128491,1.128508,0.000068,1.128491,1.128508,0.000068,-0.027230,-0.027230,2.510162e-06,195584.670984
3,-0.499998,0.913710,0.091409,1.769212,0.448276,-0.003525,-0.000843,0.000665,-0.000843,133.561797,22.654738,-0.160837,-0.230723,0.169628,-0.230723,1,367302864,547136439,0.010970,0.044343,0.186074,0.444474,0.002419,0.005634,0.002601,0.005870,60.973159,60.973159,2930.115067,2930.115067,0.011269,0.076198,0.189182,0.477827,0.003562,0.008437,0.003797,0.008594,38.295893,38.295893,...,18,-0.011914,-0.215666,0.064052,49.721306,80.747475,0.055767,31.026169,0.693546,0.691877,0.686828,0.708658,-0.000176,0.000900,0.535176,0.826308,-0.044240,-0.047569,-0.071471,-0.156174,0.111323,0.044026,0

##Scale Data (Standard Scaler)

In [6]:
#Scale data
x = data.values #returns a numpy array
standard_scaler = preprocessing.StandardScaler()
x_scaled = standard_scaler.fit_transform(x)
data_scaled_standard = pd.DataFrame(x_scaled)
data_scaled_standard.columns = data.columns

#Ad lables and metadata 
data_scaled_standard['label'] = labels.to_numpy()
data_scaled_standard['chunkID'] = chunkID.to_numpy()
data_scaled_standard['run'] = run.to_numpy()
data_scaled_standard['period'] = period.to_numpy()

data_scaled_standard.to_csv(data_folder + "all_scaled_standard.csv")
data_scaled_standard.head()

,bz,meanTPCnclF,rmsTPCnclF,meanTPCChi2,rmsTPCChi2,slopeATPCnclF,slopeCTPCnclF,slopeATPCnclFErr,slopeCTPCnclFErr,meanTPCncl,rmsTPCncl,slopeATPCncl,slopeCTPCncl,slopeATPCnclErr,slopeCTPCnclErr,hasRawQA,rawClusterCounter,rawSignalCounter,offsetdZA,slopedZA,offsetdZC,slopedZC,offsetdZAErr,slopedZAErr,offsetdZCErr,slopedZCErr,offsetdZAchi2,slopedZAchi2,offsetdZCchi2,slopedZCchi2,offsetdZAPos,slopedZAPos,offsetdZCPos,slopedZCPos,offsetdZAErrPos,slopedZAErrPos,offsetdZCErrPos,slopedZCErrPos,offsetdZAchi2Pos,slopedZAchi2Pos,...,meanMIP,meanMIPele,resolutionMIPele,electroMIPSeparation,tpcItsMatchA,tpcItsMatchHighPtA,tpcItsMatchC,tpcItsMatchHighPtC,phiPull,phiPullHighPt,ptPull,ptPullHighPt,yPull,yPullHighPt,zPull,zPullHighPt,lambdaPull,lambdaPullHighPt,tpcConstrainPhiA,tpcConstrainPhiC,meanPTRelativeA,medianPTRelativeA,rmsPTRelativeA,meanPTRelativeC,medianPTRelativeC,rmsPTRelativeC,meanHVandPTGainCorrIROC,medianHVandPTGainCorrIROC,rmsHVandPTGainCorrIROC,meanHVandPTGainCorrOROC,medianHVandPTGainCorrOROC,rmsHVandPTGainCorrOROC,meanVDriftCorr,medianVDriftCorr,rmsVDriftCorr,interactionRate,label,chunkID,run,period
0,-1.428247,0.132060,-1.195273,1.574537,-0.702346,1.404859,-1.180674,-0.211303,-1.180674,0.372801,-0.982193,1.315053,-0.886080,-0.198754,-0.886080,0.221234,-0.524265,0.335131,0.752595,0.538529,-3.805469,-3.200098,-0.325652,-0.357639,-0.271761,-0.311744,-0.085395,-0.085395,0.000873,0.000873,0.616125,0.747740,-3.423532,-2.591568,-0.278312,-0.340910,-0.254735,-0.252832,-0.693852,-0.693852,...,0.167857,0.020768,0.019287,-0.012555,0.920740,0.894499,0.982690,0.931282,0.105875,-0.279454,-0.693571,-0.390927,-1.215177,-1.147302,-1.369435,0.013329,-0.156792,-0.051477,2.091074,-2.003674,0.358170,0.357468,-0.592582,0.367008,0.366342,-0.599718,0.358164,0.357477,-0.264055,0.933084,0.932654,-0.590807,-1.769364,-1.769515,-0.714129,-0.620631,0.0,1,287000,LHC18f
1,-1.428247,0.156004,-1.107451,1.503057,-0.704743,1.287383,-1.342406,-0.230941,-1.342406,0.384322,-1.000715,1.083539,-0.909436,-0.211996,-0.909436,0.221234,-0.524265,0.335131,0.666993,-0.099785,-0.404877,-0.382826,-0.344949,-0.373592,-0.255771,-0.299517,0.327156,0.327156,0.893646,0.893646,0.260376,0.566073,-0.193846,0.114773,-0.269272,-0.344927,-0.242386,-0.241316,0.139768,0.139768,...,0.244614,0.041173,0.061838,0.006971,0.892849,0.531724,0.939828,0.925897,0.633662,1.194505,-0.420987,0.264683,0.641137,0.426363,-0.735790,-0.563810,-0.232429,1.160990,0.078069,-1.526160,0.365317,0.365014,-0.379348,0.373998,0.373690,-0.378680,0.365324,0.365082,-0.170916,0.937629,0.937482,-0.361968,-1.770066,-1.770041,-0.898776,-0.620631,0.0,2,287000,LHC18f
2,-1.428247,0.209264,-1.180419,1.532168,-0.663649,1.486528,-1.384688,-0.239118,-1.384688,0.407972,-1.027250,1.084621,-1.151679,-0.217635,-1.151679,0.221234,-0.524265,0.335131,1.489512,-0.987466,1.594846,1.468011,-0.357537,-0.389714,-0.255920,-0.305655,0.390034,0.390034,1.303380,1.303380,1.108053,-0.235408,1.749978,1.986670,-0.285598,-0.360902,-0.239145,-0.246925,1.073518,1.073518,...,-0.289517,0.004007,0.059947,-0.005501,0.952073,0.805175,0.948853,1.009140,0.179633,0.079444,-0.382609,-0.069438,0.400671,0.032698,-0.784995,-0.508842,-0.205122,0.371371,0.999531,-1.103118,0.361822,0.362705,-0.348279,0.370429,0.371317,-0.345362,0.361881,0.362698,-0.164423,0.935443,0.935968,-0.346015,-1.770806,-1.770776,-0.755877,-0.620631,0.0,3,287000,LHC18f
3,-1.428247,0.131579,-1.132418,1.614091,-0.609369,1.460009,-1.610833,-0.223359,-1.610833,0.317893,-1.038581,1.175984,-1.165258,-0.215843,-1.165258,0.221234,-0.524265,0.335131,0.944590,-0.450341,3.155897,2.850502,-0.319825,-0.351002,-0.236667,-0.279410,0.017875,0.017875,1.555480,1.555480,0.596358,0.414424,3.358282,3.437296,-0.258243,-0.319468,-0.230212,-0.230027,0.231128,0.231128,...,-0.178371,0.023577,0.050714,0.011028,0.775494,0.413610,0.783151,1.075980,-0.187720,0.796288,-0.329527,0.866984,1.090113,0.959833,-0.406908,0.032963,-0.780397,-0.581558,-0.568051,-0.876183,0.336152,0.332105,1.539635,0.344740,0.340702,1.544937,0.3362

##Scale Data To 0-1 (MinMaxScaler)

In [7]:
#Scale data
x = data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data_scaled0_1 = pd.DataFrame(x_scaled)
data_scaled0_1.columns = data.columns

#Ad lables and metadata 
data_scaled0_1['label'] = labels.to_numpy()
data_scaled0_1['chunkID'] = chunkID.to_numpy()
data_scaled0_1['run'] = run.to_numpy()
data_scaled0_1['period'] = period.to_numpy()

data_scaled0_1.to_csv(data_folder + "all_scaled0_1.csv")
data_scaled0_1.head()

,bz,meanTPCnclF,rmsTPCnclF,meanTPCChi2,rmsTPCChi2,slopeATPCnclF,slopeCTPCnclF,slopeATPCnclFErr,slopeCTPCnclFErr,meanTPCncl,rmsTPCncl,slopeATPCncl,slopeCTPCncl,slopeATPCnclErr,slopeCTPCnclErr,hasRawQA,rawClusterCounter,rawSignalCounter,offsetdZA,slopedZA,offsetdZC,slopedZC,offsetdZAErr,slopedZAErr,offsetdZCErr,slopedZCErr,offsetdZAchi2,slopedZAchi2,offsetdZCchi2,slopedZCchi2,offsetdZAPos,slopedZAPos,offsetdZCPos,slopedZCPos,offsetdZAErrPos,slopedZAErrPos,offsetdZCErrPos,slopedZCErrPos,offsetdZAchi2Pos,slopedZAchi2Pos,...,meanMIP,meanMIPele,resolutionMIPele,electroMIPSeparation,tpcItsMatchA,tpcItsMatchHighPtA,tpcItsMatchC,tpcItsMatchHighPtC,phiPull,phiPullHighPt,ptPull,ptPullHighPt,yPull,yPullHighPt,zPull,zPullHighPt,lambdaPull,lambdaPullHighPt,tpcConstrainPhiA,tpcConstrainPhiC,meanPTRelativeA,medianPTRelativeA,rmsPTRelativeA,meanPTRelativeC,medianPTRelativeC,rmsPTRelativeC,meanHVandPTGainCorrIROC,medianHVandPTGainCorrIROC,rmsHVandPTGainCorrIROC,meanHVandPTGainCorrOROC,medianHVandPTGainCorrOROC,rmsHVandPTGainCorrOROC,meanVDriftCorr,medianVDriftCorr,rmsVDriftCorr,interactionRate,label,chunkID,run,period
0,0.0,0.650378,0.212065,0.586648,0.299807,0.567304,0.506033,0.029799,0.506033,0.661859,0.130696,0.638500,0.432987,0.026360,0.432987,1.0,0.586239,0.630747,0.676500,0.505639,0.456515,0.479150,0.029047,0.025431,0.009684,0.011481,0.066360,0.066360,0.126838,0.126838,0.539923,0.675482,0.422997,0.501126,0.008516,0.015049,0.004693,0.005500,0.018955,0.018955,...,0.032376,0.539362,0.377138,0.013496,0.532074,0.573832,0.586826,0.582128,0.279488,0.558088,0.489128,0.478282,0.057793,0.397318,0.303648,0.494765,0.564013,0.819216,0.365811,0.029029,0.644374,0.643025,0.030060,0.645929,0.644593,0.029592,0.644385,0.643058,0.003333,0.731516,0.730402,0.030475,0.062052,0.062018,0.033067,0.224399,0.0,1,287000,LHC18f
1,0.0,0.652363,0.219786,0.579001,0.299619,0.553643,0.490807,0.028887,0.490807,0.662852,0.129801,0.622553,0.431335,0.025804,0.431335,1.0,0.586239,0.630747,0.672718,0.471739,0.656468,0.622429,0.028192,0.024807,0.010054,0.011773,0.098105,0.098105,0.201625,0.201625,0.526042,0.667218,0.626295,0.612202,0.008705,0.014936,0.004920,0.005719,0.078152,0.078152,...,0.033806,0.539610,0.377678,0.013927,0.529813,0.549569,0.583827,0.581842,0.295763,0.618659,0.506106,0.508984,0.096134,0.428699,0.344592,0.458420,0.559882,0.844903,0.279536,0.084570,0.645829,0.644559,0.044541,0.647352,0.646086,0.044614,0.645843,0.644604,0.005065,0.732617,0.731569,0.046314,0.061865,0.061878,0.008953,0.224399,0.0,2,287000,LHC18f
2,0.0,0.656779,0.213371,0.582115,0.302841,0.576802,0.486826,0.028507,0.486826,0.664891,0.128520,0.622628,0.414198,0.025567,0.414198,1.0,0.586239,0.630747,0.709053,0.424595,0.774050,0.716557,0.027634,0.024176,0.010050,0.011627,0.102944,0.102944,0.235948,0.235948,0.559119,0.630758,0.748653,0.689030,0.008364,0.014483,0.004980,0.005612,0.144460,0.144460,...,0.023854,0.539159,0.377654,0.013651,0.534614,0.567858,0.584458,0.586257,0.281763,0.572836,0.508496,0.493338,0.091167,0.420849,0.341413,0.461882,0.561373,0.828174,0.319029,0.133775,0.645117,0.644090,0.046651,0.646626,0.645604,0.046878,0.645142,0.644119,0.005186,0.732088,0.731203,0.047418,0.061668,0.061682,0.027615,0.224399,0.0,3,287000,LHC18f
3,0.0,0.650338,0.217591,0.590880,0.307098,0.573718,0.465535,0.029239,0.465535,0.657125,0.127972,0.628921,0.413237,0.025642,0.413237,1.0,0.586239,0.630747,0.684981,0.453121,0.865839,0.786867,0.029305,0.025690,0.010495,0.012253,0.074307,0.074307,0.257066,0.257066,0.539152,0.660319,0.849890,0.748568,0.008935,0.015657,0.005144,0.005934,0.084640,0.084640,...,0.025925,0.539396,0.377537,0.014017,0.520301,0.541670,0.572863,0.589802,0.270435,0.602295,0.511802,0.537189,0.105408,0.439337,0.365844,0.496001,0.529956,0.807985,0.251844,0.160170,0.639891,0.637870,0.174869,0.641396,0.639382,0.175343,0.639929,0.637938,0.019871,0.728152,0.726535,0.181697,0.061461,0.061460,0.007084,0.224399,0.0,4,287000,LHC18f
4,0.0,0.654262,0.221630,0.591539,0.319091,0.584242,0.494138,0.028849,0.494138,0.66

#Data Dropping

In [0]:
#Copied from Sebastian's code
DEFAULT_GROUPS_FUNCS = [
                        ('TPC-ITS_match.',  lambda col: 'tpcIts' in col),
                        ('TPC_nclusters',   lambda col: 'TPC' in col),
#                        ('TPC_nclusters',   lambda col: 'TPC' in col and 'slope' not in col),
#                        ('TPC_ncl-fits',    lambda col: 'TPC' in col and 'slope' in col),
                        ('vertex',          lambda col: 'vert' in col.lower()),
                        ('multiplicity',    lambda col: 'Mult' in col),
                        ('pT',              lambda col: ('PtA' in col or 'PtC' in col or 'qOverPt' in col) and 'tpcIts' not in col and 'deltaPt' not in col),
                        ('delta pT',        lambda col: 'deltaPt' in col),
                        ('MIP',             lambda col: ('MIP' in col)),
                        ('distr._pulls',    lambda col: 'Pull' in col),
                        ('work._conditions', lambda col: 'PTR' in col or 'HVandPT' in col or 'VDrift' in col),
                        ('DCA-fits',        lambda col: 'dcar' in col or 'dcaz' in col),
                        ('dZ-fits',         lambda col: 'dZA' in col or 'dZC' in col),
                        ('dR-fits',         lambda col: 'dRA' in col or 'dRC' in col),
                        ('random',          lambda col: 'random' in col)
                        ]

groups_funs = dict(DEFAULT_GROUPS_FUNCS)

def getCols2Drop(all_cols, groups2drop, groups_funs):
    drop = []

    for col in all_cols:
        for f in [groups_funs[key] for key in groups2drop]: # Check all groups criteria
            if f(col): #If col name fit criterium 
                drop.append(col)
                break #No need to check others if one cond is met.
    return drop

##Drop FITS

In [9]:
groups_to_drop = ['DCA-fits', 'dZ-fits', 'dR-fits']

drop = getCols2Drop(data_scaled0_1.columns, groups_to_drop, groups_funs)
data_no_fits_scaled0_1 = data_scaled0_1.drop(drop, axis=1)

data_no_fits_scaled0_1.to_csv(data_folder + "no_fits_scaled0_1.csv")
data_no_fits_scaled0_1.head()

,bz,meanTPCnclF,rmsTPCnclF,meanTPCChi2,rmsTPCChi2,slopeATPCnclF,slopeCTPCnclF,slopeATPCnclFErr,slopeCTPCnclFErr,meanTPCncl,rmsTPCncl,slopeATPCncl,slopeCTPCncl,slopeATPCnclErr,slopeCTPCnclErr,hasRawQA,rawClusterCounter,rawSignalCounter,entriesVertX,meanVertX,rmsVertX,entriesVertY,meanVertY,rmsVertY,entriesVertZ,meanVertZ,rmsVertZ,vertStatus,vertAll,vertOK,entriesMult,entriesMult.1,meanMult,rmsMult,meanMultPos,rmsMultPos,errorMultPos,meanMultNeg,rmsMultNeg,errorMultNeg,...,meanMIP,meanMIPele,resolutionMIPele,electroMIPSeparation,tpcItsMatchA,tpcItsMatchHighPtA,tpcItsMatchC,tpcItsMatchHighPtC,phiPull,phiPullHighPt,ptPull,ptPullHighPt,yPull,yPullHighPt,zPull,zPullHighPt,lambdaPull,lambdaPullHighPt,tpcConstrainPhiA,tpcConstrainPhiC,meanPTRelativeA,medianPTRelativeA,rmsPTRelativeA,meanPTRelativeC,medianPTRelativeC,rmsPTRelativeC,meanHVandPTGainCorrIROC,medianHVandPTGainCorrIROC,rmsHVandPTGainCorrIROC,meanHVandPTGainCorrOROC,medianHVandPTGainCorrOROC,rmsHVandPTGainCorrOROC,meanVDriftCorr,medianVDriftCorr,rmsVDriftCorr,interactionRate,label,chunkID,run,period
0,0.0,0.650378,0.212065,0.586648,0.299807,0.567304,0.506033,0.029799,0.506033,0.661859,0.130696,0.638500,0.432987,0.026360,0.432987,1.0,0.586239,0.630747,0.072495,0.281338,0.496442,0.072495,0.113627,0.111019,0.072495,0.314548,0.512853,0.429543,0.072211,0.072495,0.072495,0.072495,0.170502,0.217582,0.168045,0.207057,0.026479,0.171907,0.220209,0.028419,...,0.032376,0.539362,0.377138,0.013496,0.532074,0.573832,0.586826,0.582128,0.279488,0.558088,0.489128,0.478282,0.057793,0.397318,0.303648,0.494765,0.564013,0.819216,0.365811,0.029029,0.644374,0.643025,0.030060,0.645929,0.644593,0.029592,0.644385,0.643058,0.003333,0.731516,0.730402,0.030475,0.062052,0.062018,0.033067,0.224399,0.0,1,287000,LHC18f
1,0.0,0.652363,0.219786,0.579001,0.299619,0.553643,0.490807,0.028887,0.490807,0.662852,0.129801,0.622553,0.431335,0.025804,0.431335,1.0,0.586239,0.630747,0.074775,0.284612,0.499342,0.074775,0.114063,0.115167,0.074775,0.298703,0.519512,0.452418,0.074384,0.074775,0.074775,0.074775,0.171713,0.232469,0.168444,0.218634,0.026185,0.173973,0.236852,0.028232,...,0.033806,0.539610,0.377678,0.013927,0.529813,0.549569,0.583827,0.581842,0.295763,0.618659,0.506106,0.508984,0.096134,0.428699,0.344592,0.458420,0.559882,0.844903,0.279536,0.084570,0.645829,0.644559,0.044541,0.647352,0.646086,0.044614,0.645843,0.644604,0.005065,0.732617,0.731569,0.046314,0.061865,0.061878,0.008953,0.224399,0.0,2,287000,LHC18f
2,0.0,0.656779,0.213371,0.582115,0.302841,0.576802,0.486826,0.028507,0.486826,0.664891,0.128520,0.622628,0.414198,0.025567,0.414198,1.0,0.586239,0.630747,0.075070,0.281757,0.492775,0.075070,0.114695,0.119110,0.075070,0.281690,0.496006,0.429644,0.074775,0.075070,0.075069,0.075069,0.175191,0.231833,0.172669,0.222223,0.026191,0.176758,0.234255,0.028100,...,0.023854,0.539159,0.377654,0.013651,0.534614,0.567858,0.584458,0.586257,0.281763,0.572836,0.508496,0.493338,0.091167,0.420849,0.341413,0.461882,0.561373,0.828174,0.319029,0.133775,0.645117,0.644090,0.046651,0.646626,0.645604,0.046878,0.645142,0.644119,0.005186,0.732088,0.731203,0.047418,0.061668,0.061682,0.027615,0.224399,0.0,3,287000,LHC18f
3,0.0,0.650338,0.217591,0.590880,0.307098,0.573718,0.465535,0.029239,0.465535,0.657125,0.127972,0.628921,0.413237,0.025642,0.413237,1.0,0.586239,0.630747,0.073988,0.279202,0.497806,0.073988,0.110782,0.113392,0.073988,0.291844,0.484649,0.428204,0.073704,0.073988,0.073987,0.073987,0.172629,0.221589,0.169854,0.207549,0.026137,0.174482,0.229296,0.028253,...,0.025925,0.539396,0.377537,0.014017,0.520301,0.541670,0.572863,0.589802,0.270435,0.602295,0.511802,0.537189,0.105408,0.439337,0.365844,0.496001,0.529956,0.807985,0.251844,0.160170,0.639891,0.637870,0.174869,0.641396,0.639382,0.175343,0.639929,0.637938,0.019871,0.728152,0.726535,0.181697,0.061461,0.061460,0.007084,0.224399,0.0,4,287000,LHC18f
4,0.0,0.654262,0.221630,0.591539,0.319091,0.584242,0.494138,0.028849,0.494138,0.663947,0.132655,0.660335,0.417028,0.025590,0.417028,

##Drop RFE

In [0]:
#Get ranking
%%capture
!wget 'https://drive.google.com/uc?export=download&id=1VniFAb_J0uHS0PCZMvd7JCEWtX3O8V3t' -O feature_rank.csv

In [11]:
ranks = pd.read_csv('feature_rank.csv',index_col=0)
rfe = ranks['logreg']
rfe = rfe[:120].to_numpy() #Get worse 100 features by RFE

data_no_rfe_scaled0_1 = data_scaled0_1.drop(rfe, axis=1)

data_no_rfe_scaled0_1.to_csv(data_folder + "no_120logreg_rfe_scaled0_1.csv")
data_no_rfe_scaled0_1.head()

,meanTPCnclF,meanTPCChi2,rmsTPCChi2,slopeATPCnclF,slopeCTPCnclF,slopeCTPCnclFErr,meanTPCncl,rmsTPCncl,slopeCTPCncl,slopeCTPCnclErr,hasRawQA,rawSignalCounter,slopedZA,offsetdZC,slopedZC,offsetdZAErr,slopedZAErr,offsetdZAchi2,slopedZAchi2,offsetdZAchi2Pos,slopedZAchi2Pos,offsetdZCchi2Pos,slopedZCchi2Pos,slopedZANeg,offsetdZCNeg,slopedZCNeg,offsetdZAchi2Neg,slopedZAchi2Neg,offsetdRC,offsetdRAchi2,slopedRAchi2,offsetdRCchi2,slopedRCchi2,dcarCP0,dcarCP1,offsetdRAchi2Pos,slopedRAchi2Pos,offsetdRCchi2Pos,slopedRCchi2Pos,offsetdRCchi2Neg,...,dcar_posC_0_Err,dcar_posC_1_Err,dcar_posC_2_Err,dcar_negA_1,dcar_negA_chi2,dcar_negA_0_Err,dcar_negA_1_Err,dcar_negA_2_Err,dcaz_negA_0,dcaz_negA_1,dcaz_negA_0_Err,dcaz_negA_1_Err,dcaz_negA_2_Err,dcaz_negC_chi2,dcaz_negC_0_Err,dcaz_negC_1_Err,dcaz_negC_2_Err,dcar_negC_chi2,dcar_negC_0_Err,dcar_negC_1_Err,dcar_negC_2_Err,iroc_A_side,oroc_A_side,iroc_C_side,oroc_C_side,tpcItsMatchA,tpcItsMatchC,phiPullHighPt,ptPull,ptPullHighPt,zPullHighPt,lambdaPull,medianPTRelativeA,medianHVandPTGainCorrIROC,meanVDriftCorr,rmsVDriftCorr,label,chunkID,run,period
0,0.650378,0.586648,0.299807,0.567304,0.506033,0.506033,0.661859,0.130696,0.432987,0.432987,1.0,0.630747,0.505639,0.456515,0.479150,0.029047,0.025431,0.066360,0.066360,0.018955,0.018955,0.103951,0.103951,0.347511,0.486962,0.366647,0.135214,0.135214,0.104897,0.041065,0.041065,0.165264,0.165264,0.012700,0.863747,0.017025,0.017025,0.166519,0.166519,0.169389,...,0.063047,0.064405,0.062340,0.642167,0.040081,0.089056,0.082068,0.091172,0.387952,0.529658,0.075152,0.073086,0.076870,0.049049,0.064750,0.062988,0.066403,0.096599,0.092639,0.089595,0.096029,1.0,1.0,1.0,1.0,0.532074,0.586826,0.558088,0.489128,0.478282,0.494765,0.564013,0.643025,0.643058,0.062052,0.033067,0.0,1,287000,LHC18f
1,0.652363,0.579001,0.299619,0.553643,0.490807,0.490807,0.662852,0.129801,0.431335,0.431335,1.0,0.630747,0.471739,0.656468,0.622429,0.028192,0.024807,0.098105,0.098105,0.078152,0.078152,0.195836,0.195836,0.318030,0.662324,0.531044,0.126243,0.126243,0.184002,0.120756,0.120756,0.277622,0.277622,0.013681,0.864576,0.005406,0.005406,0.270753,0.270753,0.295496,...,0.061939,0.063179,0.061326,0.648996,0.029575,0.088029,0.080961,0.090309,0.368447,0.537606,0.074312,0.072295,0.075983,0.053690,0.062965,0.061329,0.064488,0.080482,0.090666,0.087815,0.093847,1.0,1.0,1.0,1.0,0.529813,0.583827,0.618659,0.506106,0.508984,0.458420,0.559882,0.644559,0.644604,0.061865,0.008953,0.0,2,287000,LHC18f
2,0.656779,0.582115,0.302841,0.576802,0.486826,0.486826,0.664891,0.128520,0.414198,0.414198,1.0,0.630747,0.424595,0.774050,0.716557,0.027634,0.024176,0.102944,0.102944,0.144460,0.144460,0.224404,0.224404,0.301996,0.760046,0.627993,0.067836,0.067836,0.229955,0.088526,0.088526,0.349164,0.349164,0.013235,0.864001,0.014864,0.014864,0.332814,0.332814,0.391960,...,0.061592,0.062751,0.061049,0.628015,0.040652,0.088169,0.080976,0.090568,0.377793,0.560263,0.073681,0.071540,0.075483,0.070414,0.062983,0.061441,0.064412,0.067023,0.090231,0.087507,0.093285,1.0,1.0,1.0,1.0,0.534614,0.584458,0.572836,0.508496,0.493338,0.461882,0.561373,0.644090,0.644119,0.061668,0.027615,0.0,3,287000,LHC18f
3,0.650338,0.590880,0.307098,0.573718,0.465535,0.465535,0.657125,0.127972,0.413237,0.413237,1.0,0.630747,0.453121,0.865839,0.786867,0.029305,0.025690,0.074307,0.074307,0.084640,0.084640,0.224069,0.224069,0.307199,0.833085,0.699755,0.059567,0.059567,0.268384,0.103972,0.103972,0.427887,0.427887,0.016565,0.863877,0.016920,0.016920,0.391857,0.391857,0.491013,...,0.062720,0.063980,0.062087,0.648159,0.024053,0.088482,0.081358,0.090786,0.376294,0.584689,0.074933,0.072717,0.076810,0.095369,0.064623,0.062784,0.066359,0.065335,0.091349,0.088426,0.094600,1.0,1.0,1.0,1.0,0.520301,0.572863,0.602295,0.511802,0.537189,0.496001,0.529956,0.637870,0.637938,0.061461,0.007084,0.0,4,287000,LHC18f
4,0.654262,0.591539,0.319091,0.584242,0.494138,0.494138,0.663947,0.132655,0.417028,0.417028,1.0,0.630747,0.486288,0.710244,0.684728,0.027425,0.023841,0.073414,0.073414,0.

In [12]:
#Get final features as 1D list - crazy one liner
final_features = pd.read_csv('/content/gdrive/My Drive/ColabNotebooks/Analysis/final_features_list.csv', squeeze=True, header=None, index_col=0).to_numpy().squeeze().tolist()
metadata = ['label', 'chunkID', 'run', 'period']
data_final_feat_scaled0_1 = data_scaled0_1[final_features + metadata]

data_final_feat_scaled0_1.to_csv(data_folder + "data_final_feat_scaled0_1.csv")
print(data_final_feat_scaled0_1.shape)
data_final_feat_scaled0_1.head()

(3429, 132)


,dcar_posA_2_Err,slopedZAchi2Pos,MIPattachSlopeC,meanMultNeg,mediumPtAPos,dcarCP0,dcaz_negA_1_Err,tpcConstrainPhiA,dcar_negA_chi2,offsetdZC,dcaz_negA_0_Err,zPullHighPt,vertOK,tpcItsMatchHighPtA,deltaPt,resolutionMIP,entriesVertY,entriesMult,dcaz_posA_0,dcar_posC_1,slopedRAchi2Pos,tpcConstrainPhiC,iroc_A_side,meanVertZ,slopedZAchi2,deltaPtC,offsetdZAErr,offsetdZA,dcaz_posA_2_Err,dcaz_negC_0_Err,offsetdRA,meanMIPele,offsetdZAchi2Pos,medianHVandPTGainCorrIROC,iroc_C_side,vertAll,dcar_negA_1_Err,offsetdRAchi2Pos,meanMult,qOverPtC,...,meanTPCChi2,rmsVertY,tpcItsMatchHighPtC,dcaz_negC_2_Err,yPullHighPt,tpcItsMatchC,deltaPtchi2C,deltaPtchi2,rmsMultNeg,meanTPCnclF,ptPullHighPt,oroc_C_side,slopedZCchi2Pos,rmsTPCncl,dcar_posC_2_Err,offsetdZCchi2Pos,deltaPtchi2A,resolutionMIPele,dcarCP1,dcar_posA_1_Err,slopedRCchi2Pos,slopeCTPCnclFErr,slopeCTPCnclF,dcar_negC_0_Err,slopedZC,lambdaPull,dcaz_posC_1_Err,offsetdRCchi2Neg,dcaz_negC_chi2,offsetdRCchi2Pos,dcaz_posC_0_Err,oroc_A_side,offsetdZAchi2,dcar_negA_0_Err,dcar_negC_chi2,slopeATPCnclF,label,chunkID,run,period
0,0.084978,0.018955,0.842762,0.171907,0.269421,0.012700,0.073086,0.365811,0.040081,0.456515,0.075152,0.494765,0.072495,0.573832,0.778597,0.094750,0.072495,0.072495,0.402053,0.832478,0.017025,0.029029,1.0,0.314548,0.066360,0.607326,0.029047,0.676500,0.080796,0.064750,0.827629,0.539362,0.018955,0.643058,1.0,0.072211,0.082068,0.017025,0.170502,0.999787,...,0.586648,0.111019,0.582128,0.066403,0.397318,0.586826,0.232100,0.210562,0.220209,0.650378,0.478282,1.0,0.103951,0.130696,0.062340,0.103951,0.254022,0.377138,0.863747,0.091667,0.166519,0.506033,0.506033,0.092639,0.479150,0.564013,0.063305,0.169389,0.049049,0.166519,0.060923,1.0,0.066360,0.089056,0.096599,0.567304,0.0,1,287000,LHC18f
1,0.083858,0.078152,0.828729,0.173973,0.240232,0.013681,0.072295,0.279536,0.029575,0.656468,0.074312,0.458420,0.074775,0.549569,0.745472,0.091434,0.074775,0.074775,0.388408,0.802137,0.005406,0.084570,1.0,0.298703,0.098105,0.597697,0.028192,0.672718,0.079918,0.062965,0.794622,0.539610,0.078152,0.644604,1.0,0.074384,0.080961,0.005406,0.171713,0.999787,...,0.579001,0.115167,0.581842,0.064488,0.428699,0.583827,0.149810,0.385217,0.236852,0.652363,0.508984,1.0,0.195836,0.129801,0.061326,0.195836,0.515967,0.377678,0.864576,0.090594,0.270753,0.490807,0.490807,0.090666,0.622429,0.559882,0.061817,0.295496,0.053690,0.270753,0.059619,1.0,0.098105,0.088029,0.080482,0.553643,0.0,2,287000,LHC18f
2,0.083916,0.144460,0.829774,0.176758,0.276467,0.013235,0.071540,0.319029,0.040652,0.774050,0.073681,0.461882,0.075070,0.567858,0.739555,0.089755,0.075070,0.075069,0.394434,0.787007,0.014864,0.133775,1.0,0.281690,0.102944,0.589850,0.027634,0.709053,0.079374,0.062983,0.800667,0.539159,0.144460,0.644119,1.0,0.074775,0.080976,0.014864,0.175191,0.999787,...,0.582115,0.119110,0.586257,0.064412,0.420849,0.584458,0.421557,0.435664,0.234255,0.656779,0.493338,1.0,0.224404,0.128520,0.061049,0.224404,0.247578,0.377654,0.864001,0.090476,0.332814,0.486826,0.486826,0.090231,0.716557,0.561373,0.061571,0.391960,0.070414,0.332814,0.059297,1.0,0.102944,0.088169,0.067023,0.576802,0.0,3,287000,LHC18f
3,0.085152,0.084640,0.835028,0.174482,0.258437,0.016565,0.072717,0.251844,0.024053,0.865839,0.074933,0.496001,0.073988,0.541670,0.761264,0.091761,0.073988,0.073987,0.389906,0.784608,0.016920,0.160170,1.0,0.291844,0.074307,0.600640,0.029305,0.684981,0.080639,0.064623,0.779825,0.539396,0.084640,0.637938,1.0,0.073704,0.081358,0.016920,0.172629,0.999787,...,0.590880,0.113392,0.589802,0.066359,0.439337,0.572863,0.565959,0.406805,0.229296,0.650338,0.537189,1.0,0.224069,0.127972,0.062087,0.224069,0.425016,0.377537,0.863877,0.091592,0.391857,0.465535,0.465535,0.091349,0.786867,0.529956,0.063318,0.491013,0.095369,0.391857,0.060874,1.0,0.074307,0.088482,0.065335,0.573718,0.0,4,287000,LHC18f
4,0.083390,0.070673,0.833388,0.174964,0.265868,0.016429,0.071677,0.286252,0.048721,0.710244,0.073648,0.502829,0.075331,0.544351,0.762631,0.092249,0.075331,0.075331,0.400363,0.81